<a href="https://colab.research.google.com/github/nkyc-no-name/Llama3_Korean/blob/main/create_LINA_MMLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set up your OpenAI API key
import os
from openai import OpenAI

client = OpenAI(api_key="sk-")  # Pass the key directly

In [ ]:
def create_mcq_from_qa(qa_pair):
    # Extract question and answer from the QA pair
    parts = qa_pair.split('[/INST]')
    question = parts[0].strip()[6:]  # Removing '[INST] ' from the start
    answer = parts[1].strip()

    # Prompt for GPT-4 to generate multiple-choice options
    prompt = f"I need to create multiple-choice evaluation problems in Korean for a Question-Answer pair I provide you. \
    e.g. Question-Answer pair: \
    Q: 지난달 통장잔고가 부족해서 자동이체 출금이 되지 않았습니다. 추후 입금했는데 언제 출금이 되나요? \
    A: 지정하신 정규 이체일에 출금 완료되지 않을 경우 회사에서 정한 기준에 따라 보험료 정상출금 시까지 재청구 진행됩니다.\
    단,실효기준일 이전까지만 진행되며,자세한 재청구 기준 확인이 필요하시거나 잔고 유지 후5영업일 이상 경과했을 경우 등 문의가 있으신 경우 당사 고객센터(1588-0058)로 연락주시기 바랍니다. \
    The format of the output problem should be like this: \
    e.g. Q: 지난달 통장잔고가 부족해서 자동이체 출금이 안 될 경우, 출금이 재청구되는 기준은 무엇인가요? \
    1. 보험료 정상출금 시까지, 2. 통장잔고가 0이 될 때까지, 3. 고객센터에 문의한 후, 4. 5영업일 이후, \
    정답: B. \
    If the question in the Question-Answer pair includes a specific insurance product, \
    make sure to refer to the name in the beginnig of the question of each problem. \
    Any comment, summary, explanation should not be generated other than requested format. \
    Question: {question}\nAnswer: {answer}\n"

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt},
        ]
    )

    return response.choices[0].message.content.strip()

def process_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        qa_lines = infile.readlines()

    all_mcqs = []

    for qa_line in qa_lines:
        if qa_line.strip():  # Skip empty lines
            mcq = create_mcq_from_qa(qa_line.strip())
            all_mcqs.append(mcq)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for mcq in all_mcqs:
            outfile.write(mcq + '\n')

# Specify the input and output file paths
input_file = 'all.txt'
output_file = 'lina_mmlu.txt'

# Process the file to generate multiple-choice questions
process_file(input_file, output_file)

print(f'Multiple-choice questions have been written to {output_file}')

Multiple-choice questions have been written to lina_mmlu.txt


In [ ]:
import csv

# 입력 파일과 출력 파일 경로
input_file = 'lina_mmlu.txt'
output_file = 'lina_mmlu.csv'

# 결과를 저장할 리스트
data = []

# 파일 읽기
with open(input_file, 'r', encoding='utf-8') as file:
    while True:
        line = file.readline()
        if not line:
            break
        if line.startswith("Q:"):
            question = line.strip()[3:]
            choices = []
            for _ in range(4):
                choice_line = file.readline().strip()
                choices.append(choice_line[3:].strip())
            answer_line = file.readline().strip()
            if "정답:" in answer_line:
                answer = answer_line.split("정답:")[1].strip().rstrip('.')  # 정답 문자 추출
            else:
                answer = ""
            data.append([question, choices[0], choices[1], choices[2], choices[3], answer])

# CSV 파일로 쓰기
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Question', 'A', 'B', 'C', 'D', 'Answer'])  # 헤더 작성
    for row in data:
        writer.writerow(row)

print("CSV 파일로 변환이 완료되었습니다.")
